In [11]:
import csv
from django.db import connection
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from batid.models import Building
# from django.contrib.gis.geos import GEOSGeometry
# from django.contrib.gis.db.models.functions import Distance
import pandas as pd

df = pd.DataFrame()
with open('EFA_Operat.csv', newline='') as csvfile:
    df = pd.read_csv(csvfile, sep=';')

/tmp/ipykernel_4507/948396588.py:12: DtypeWarning: Columns (2,6,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csvfile, sep=';')


In [12]:
print("nombre de lignes")
print(len(df))

nombre de lignes
294047


In [17]:
print("nombre de lignes pour lesquelles le geocodage renvoie un résultat")
print(len(df[df['result_status'] == "ok"]))

print("nombre de lignes pour lesquelles le geocodage ne renvoie pas de résultat")
print(len(df[df['result_status'] != "ok"]))

print("nombre de résultats de type housenumber")
print(len(df[df['result_type'] == "housenumber"]))

print("nombre de résultats de type housenumber et précision > 0.75")
print(len(df[(df['result_type'] == "housenumber") & (df['result_score'] > 0.75)]))

nombre de lignes pour lesquelles le geocodage renvoie un résultat
292467
nombre de lignes pour lesquelles le geocodage ne renvoie pas de résultat
1580
nombre de résultats de type housenumber
181633
nombre de résultats de type housenumber et précision > 0.75
126951


In [19]:
# on garde les geocodages de qualité
df_qual = df[(df['result_type'] == "housenumber") & (df['result_score'] > 0.75)]

df_requests = df_qual['result_id'].apply(lambda x: f"select * from batid_building bb left join batid_building_addresses bba on bba.building_id  = bb.id where bba.address_id = '{x}';")

# write the result in a csv file
df_requests.to_csv('requests.csv', index=False)